## LIMPIEZA DE DATOS ARCHIVOS CSV:


(Pendiente explicación de lo que contienen los archivos)

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('../src')
from suport import unif_col
from suport import lower_tildes
from suport import clean_year
from suport import trimestre
from suport import eliminar_alfa  
from suport import ccaa
from suport import comunidades
from suport import eliminar_num
from suport import provincias
from suport import crear_ccaa
from unidecode import unidecode
pd.set_option('display.max_columns', None) 
from fuzzywuzzy import process
from fuzzywuzzy import fuzz


### 1. LIMPIEZA ARCHIVO DENUNCIAS POR VIOLENCIA DE GÉNERO:


In [2]:
denu = pd.read_csv('../data/denuncias.csv')
denu.head()

,Año,Trimestre,Origen de la denuncia,Almería,Cádiz,Córdoba,Granada,Huelva,Jaén,Málaga,Sevilla,Huesca,Teruel,Zaragoza,Asturias,Illes Balears,Las Palmas,Santa Cruz de Tenerife,Cantabria,Ávila,Burgos,León,Palencia,Salamanca,Segovia,Soria,Valladolid,Zamora,Albacete,Ciudad Real,Cuenca,Guadalajara,Toledo,Barcelona,Girona,Lleida,Tarragona,Alicante/Alacant,Castellón/Castelló,Valencia/València,Badajoz,Cáceres,A Coruña,Lugo,Ourense,Pontevedra,Madrid,Murcia,Navarra,Araba/Álava,Gipuzkoa,Bizkaia,La Rioja,Ceuta,Melilla
0,Año 2009,Primero,Presentada directamente por victima,40.0,140.0,11.0,111.0,14.0,17.0,100.0,126.0,7.0,30.0,19.0,9.0,137.0,11.0,130.0,44.0,7.0,24.0,20.0,2.0,120.0,3.0,0.0,9.0,11.0,21.0,27.0,3.0,57.0,103.0,173.0,37.0,15.0,212.0,57.0,9.0,327.0,70.0,107.0,17.0,15.0,19.0,83.0,252.0,58.0,5.0,9.0,24.0,34.0,51.0,0.0,2.0
1,Año 2009,Primero,Presentada directamente por familiares,1.0,17.0,0.0,0.0,0.0,0.0,4.0,8.0,4.0,0.0,1.0,1.0,17.0,3.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,4.0,0.0,0.0,2.0,3.0,0.0,0.0,1.0,0.0,3.0,1.0,0.0,2.0,11.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Año 2009,Primero,Atestados policiales - con denuncia victima,419.0,549.0,307.0,748.0,437.0,313.0,837.0,731.0,63.0,9.0,409.0,427.0,650.0,828.0,624.0,158.0,71.0,148.0,138.0,68.0,22.0,47.0,38.0,283.0,57.0,176.0,176.0,85.0,209.0,201.0,1929.0,378.0,202.0,271.0,1159.0,189.0,898.0,104.0,89.0,298.0,76.0,172.0,453.0,4089.0,958.0,187.0,173.0,230.0,397.0,100.0,57.0,61.0
3,Año 2009,Primero,Atestados policiales - con denuncia familiar,7.0,8.0,6.0,4.0,0.0,7.0,28.0,3.0,0.0,0.0,8.0,3.0,15.0,8.0,5.0,18.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,5.0,14.0,0.0,0.0,1.0,39.0,1.0,0.0,11.0,21.0,8.0,2.0,0.0,0.0,9.0,1.0,2.0,1.0,25.0,27.0,9.0,1.0,2.0,19.0,2.0,0.0,0.0
4,Año 2009,Primero,Atestados policiales - por intervención direct...,82.0,30.0,13.0,15.0,25.0,30.0,210.0,38.0,8.0,0.0,72.0,20.0,107.0,91.0,43.0,28.0,17.0,33.0,47.0,3.0,3.0,11.0,1.0,77.0,0.0,20.0,25.0,0.0,25.0,21.0,535.0,96.0,8.0,115.0,267.0,51.0,780.0,7.0,10.0,32.0,8.0,3.0,0.0,831.0,175.0,36.0,5.0,12.0,71.0,34.0,7.0,9.0


In [3]:
denu.columns = unif_col(denu.columns)
denu['trimestre'] = lower_tildes(denu['trimestre'])
denu.isna().value_counts()

ano    trimestre  origen de la denuncia  almeria  cadiz  cordoba  granada  huelva  jaen   malaga  sevilla  huesca  teruel  zaragoza  asturias  illes balears  las palmas  santa cruz de tenerife  cantabria  avila  burgos  leon   palencia  salamanca  segovia  soria  valladolid  zamora  albacete  ciudad real  cuenca  guadalajara  toledo  barcelona  girona  lleida  tarragona  alicante/alacant  castellon/castello  valencia/valencia  badajoz  caceres  a coruna  lugo   ourense  pontevedra  madrid  murcia  navarra  araba/alava  gipuzkoa  bizkaia  la rioja  ceuta  melilla
False  False      False                  False    False  False    False    False   False  False   False    False   False   False     False     False          False       False                   False      False  False   False  False     False      False    False  False       False   False     False        False   False        False   False      False   False   False      False             False               False              

In [4]:
denu.rename(columns={'ano': 'año'}, inplace=True)
denu.año = clean_year(denu.año)

In [5]:
denu.rename(columns={'origen de la denuncia': 'origen_denuncia'}, inplace=True)
denu['origen_denuncia'] = lower_tildes(denu['origen_denuncia'])

In [6]:
denu['origen_denuncia'].unique()

array(['presentada directamente por victima',
       'presentada directamente por familiares',
       'atestados policiales - con denuncia victima',
       'atestados policiales - con denuncia familiar',
       'atestados policiales - por intervencion directa policial',
       'parte de lesiones', 'servicios asistencia-terceros  en general'],
      dtype=object)

In [7]:
origen = {'presentada directamente por victima': 'victima',
          'presentada directamente por familiares': 'familiar',
          'atestados policiales - con denuncia victima': 'policia_victima',
          'atestados policiales - con denuncia familiar': 'policia_familiar',
          'atestados policiales - por intervencion directa policial': 'intervencion_policial_directa',
          'parte de lesiones': 'parte_lesiones',
          'servicios asistencia-terceros  en general': 'asistencia_terceros'}

denu['origen_denuncia'] = denu['origen_denuncia'].map(origen)

In [8]:
col_int = denu.columns[3:]

denu[col_int] = denu[col_int].astype(int)

In [9]:
denu = pd.melt(denu, id_vars=['año', 'trimestre', 'origen_denuncia'], 
                var_name='provincia', value_name='total_denuncias')

In [10]:
denu

,año,trimestre,origen_denuncia,provincia,total_denuncias
0,2009,primero,victima,almeria,40
1,2009,primero,familiar,almeria,1
2,2009,primero,policia_victima,almeria,419
3,2009,primero,policia_familiar,almeria,7
4,2009,primero,intervencion_policial_directa,almeria,82
...,...,...,...,...,...
21107,2023,segundo,policia_victima,melilla,80
21108,2023,segundo,policia_familiar,melilla,1
21109,2023,segundo,intervencion_policial_directa,melilla,7
21110,2023,segundo,parte_lesiones,melilla,5


In [11]:
denu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21112 entries, 0 to 21111
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   año              21112 non-null  int32 
 1   trimestre        21112 non-null  object
 2   origen_denuncia  21112 non-null  object
 3   provincia        21112 non-null  object
 4   total_denuncias  21112 non-null  int64 
dtypes: int32(1), int64(1), object(3)
memory usage: 742.3+ KB


### 2. LIMPIEZA ARCHIVO LLAMADAS 016:

In [12]:
llam = pd.read_csv('../data/llamadas.csv')
llam.head()

,Año,Trimestre,Llamante,Almería,Cádiz,Córdoba,Granada,Huelva,Jaén,Málaga,Sevilla,Huesca,Teruel,Zaragoza,Asturias,Illes Balears,Las Palmas,Santa Cruz de Tenerife,Cantabria,Ávila,Burgos,León,Palencia,Salamanca,Segovia,Soria,Valladolid,Zamora,Albacete,Ciudad Real,Cuenca,Guadalajara,Toledo,Barcelona,Girona,Lleida,Tarragona,Alicante/Alacant,Castellón/Castelló,Valencia/València,Badajoz,Cáceres,A Coruña,Lugo,Ourense,Pontevedra,Madrid,Murcia,Navarra,Araba/Álava,Bizkaia,Gipuzkoa,La Rioja,Ceuta,Melilla,No consta
0,Año 2007,Tercero,Usuaria,3.0,2.0,7.0,6.0,4.0,NaN,8.0,16.0,1.0,NaN,4.0,5.0,6.0,20.0,6.0,8.0,3.0,NaN,2.0,2.0,1.0,NaN,1.0,2.0,NaN,NaN,NaN,NaN,1.0,1.0,27.0,3.0,1.0,4.0,8.0,2.0,4.0,3.0,4.0,5.0,NaN,2.0,2.0,99.0,6.0,4.0,1.0,2.0,1.0,NaN,1.0,1.0,16.0
1,Año 2007,Tercero,Familiares/Personas allegadas,NaN,NaN,3.0,1.0,2.0,NaN,5.0,9.0,NaN,NaN,NaN,2.0,1.0,3.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,1.0,5.0,NaN,NaN,1.0,5.0,NaN,4.0,2.0,1.0,1.0,NaN,NaN,1.0,30.0,1.0,NaN,NaN,2.0,NaN,NaN,NaN,1.0,3.0
2,Año 2007,Tercero,Otros,NaN,NaN,NaN,1.0,2.0,2.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,4.0,2.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,20.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,3.0
3,Año 2007,Cuarto,Usuaria,150.0,187.0,155.0,250.0,82.0,131.0,357.0,608.0,37.0,40.0,152.0,248.0,309.0,365.0,298.0,177.0,60.0,65.0,128.0,44.0,80.0,48.0,25.0,159.0,36.0,50.0,93.0,34.0,35.0,180.0,1229.0,90.0,83.0,134.0,498.0,102.0,592.0,160.0,78.0,317.0,56.0,64.0,157.0,3368.0,386.0,123.0,62.0,195.0,76.0,49.0,22.0,18.0,123.0
4,Año 2007,Cuarto,Familiares/Personas allegadas,27.0,44.0,26.0,41.0,17.0,17.0,78.0,135.0,8.0,2.0,35.0,51.0,43.0,71.0,49.0,23.0,12.0,10.0,16.0,8.0,25.0,1.0,8.0,22.0,2.0,13.0,23.0,7.0,9.0,36.0,247.0,24.0,15.0,21.0,75.0,21.0,125.0,26.0,19.0,45.0,9.0,1.0,33.0,604.0,61.0,32.0,16.0,38.0,19.0,12.0,2.0,3.0,28.0


In [13]:
llam.columns = unif_col(llam.columns)
llam['trimestre'] = lower_tildes(llam['trimestre'])
llam.llamante = lower_tildes(llam.llamante)
llam.rename(columns={'ano': 'año'}, inplace=True)
llam.año = clean_year(llam.año)

In [14]:
llam.llamante.unique()

array(['usuaria', 'familiares/personas allegadas', 'otros', 'desconocido',
       'no consta'], dtype=object)

In [15]:
llam.columns

Index(['año', 'trimestre', 'llamante', 'almeria', 'cadiz', 'cordoba',
       'granada', 'huelva', 'jaen', 'malaga', 'sevilla', 'huesca', 'teruel',
       'zaragoza', 'asturias', 'illes balears', 'las palmas',
       'santa cruz de tenerife', 'cantabria', 'avila', 'burgos', 'leon',
       'palencia', 'salamanca', 'segovia', 'soria', 'valladolid', 'zamora',
       'albacete', 'ciudad real', 'cuenca', 'guadalajara', 'toledo',
       'barcelona', 'girona', 'lleida', 'tarragona', 'alicante/alacant',
       'castellon/castello', 'valencia/valencia', 'badajoz', 'caceres',
       'a coruna', 'lugo', 'ourense', 'pontevedra', 'madrid', 'murcia',
       'navarra', 'araba/alava', 'bizkaia', 'gipuzkoa', 'la rioja', 'ceuta',
       'melilla', 'no consta'],
      dtype='object')

In [16]:
quien_llama = {'usuaria': 'usuaria',
               'familiares/personas allegadas': 'familiar',
               'otros': 'desconocido',
               'desconocido': 'desconocido',
               'no Consta': 'desconocido'}

llam['llamante'] = llam['llamante'].map(quien_llama)

In [17]:
llam.isna().value_counts()

año    trimestre  llamante  almeria  cadiz  cordoba  granada  huelva  jaen   malaga  sevilla  huesca  teruel  zaragoza  asturias  illes balears  las palmas  santa cruz de tenerife  cantabria  avila  burgos  leon   palencia  salamanca  segovia  soria  valladolid  zamora  albacete  ciudad real  cuenca  guadalajara  toledo  barcelona  girona  lleida  tarragona  alicante/alacant  castellon/castello  valencia/valencia  badajoz  caceres  a coruna  lugo   ourense  pontevedra  madrid  murcia  navarra  araba/alava  bizkaia  gipuzkoa  la rioja  ceuta  melilla  no consta
False  False      False     False    False  False    False    False   False  False   False    False   False   False     False     False          False       False                   False      False  False   False  False     False      False    False  False       False   False     False        False   False        False   False      False   False   False      False             False               False              False    False 

In [18]:
llam.fillna(0, inplace = True)

llam.head()

,año,trimestre,llamante,almeria,cadiz,cordoba,granada,huelva,jaen,malaga,sevilla,huesca,teruel,zaragoza,asturias,illes balears,las palmas,santa cruz de tenerife,cantabria,avila,burgos,leon,palencia,salamanca,segovia,soria,valladolid,zamora,albacete,ciudad real,cuenca,guadalajara,toledo,barcelona,girona,lleida,tarragona,alicante/alacant,castellon/castello,valencia/valencia,badajoz,caceres,a coruna,lugo,ourense,pontevedra,madrid,murcia,navarra,araba/alava,bizkaia,gipuzkoa,la rioja,ceuta,melilla,no consta
0,2007,tercero,usuaria,3.0,2.0,7.0,6.0,4.0,0.0,8.0,16.0,1.0,0.0,4.0,5.0,6.0,20.0,6.0,8.0,3.0,0.0,2.0,2.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,27.0,3.0,1.0,4.0,8.0,2.0,4.0,3.0,4.0,5.0,0.0,2.0,2.0,99.0,6.0,4.0,1.0,2.0,1.0,0.0,1.0,1.0,16.0
1,2007,tercero,familiar,0.0,0.0,3.0,1.0,2.0,0.0,5.0,9.0,0.0,0.0,0.0,2.0,1.0,3.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,1.0,5.0,0.0,4.0,2.0,1.0,1.0,0.0,0.0,1.0,30.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,3.0
2,2007,tercero,desconocido,0.0,0.0,0.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,4.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,20.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,3.0
3,2007,cuarto,usuaria,150.0,187.0,155.0,250.0,82.0,131.0,357.0,608.0,37.0,40.0,152.0,248.0,309.0,365.0,298.0,177.0,60.0,65.0,128.0,44.0,80.0,48.0,25.0,159.0,36.0,50.0,93.0,34.0,35.0,180.0,1229.0,90.0,83.0,134.0,498.0,102.0,592.0,160.0,78.0,317.0,56.0,64.0,157.0,3368.0,386.0,123.0,62.0,195.0,76.0,49.0,22.0,18.0,123.0
4,2007,cuarto,familiar,27.0,44.0,26.0,41.0,17.0,17.0,78.0,135.0,8.0,2.0,35.0,51.0,43.0,71.0,49.0,23.0,12.0,10.0,16.0,8.0,25.0,1.0,8.0,22.0,2.0,13.0,23.0,7.0,9.0,36.0,247.0,24.0,15.0,21.0,75.0,21.0,125.0,26.0,19.0,45.0,9.0,1.0,33.0,604.0,61.0,32.0,16.0,38.0,19.0,12.0,2.0,3.0,28.0


In [19]:
col_int = llam.columns[3:]

llam[col_int] = llam[col_int].astype(int)

In [20]:
llam = pd.melt(llam, id_vars=['año', 'trimestre', 'llamante'], 
                var_name='provincia', value_name='total_llamadas')

In [21]:
llam.head()

,año,trimestre,llamante,provincia,total_llamadas
0,2007,tercero,usuaria,almeria,3
1,2007,tercero,familiar,almeria,0
2,2007,tercero,desconocido,almeria,0
3,2007,cuarto,usuaria,almeria,150
4,2007,cuarto,familiar,almeria,27


In [22]:
llam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12508 entries, 0 to 12507
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   año             12508 non-null  int32 
 1   trimestre       12508 non-null  object
 2   llamante        12508 non-null  object
 3   provincia       12508 non-null  object
 4   total_llamadas  12508 non-null  int64 
dtypes: int32(1), int64(1), object(3)
memory usage: 439.9+ KB


### 3. LIMPIEZA ARCHIVO ORDEN PROTECCIÓN

In [23]:
ord = pd.read_csv('../data/orden_proteccion.csv')
ord.head()

,Año,Trimestre,Provincia,Número de órdenes de protección
0,Año 2009,Primero,Almería,82.0
1,Año 2009,Primero,Cádiz,312.0
2,Año 2009,Primero,Córdoba,125.0
3,Año 2009,Primero,Granada,163.0
4,Año 2009,Primero,Huelva,138.0


In [24]:
ord.columns = unif_col(ord.columns)
ord.rename(columns={'ano': 'año'}, inplace=True)
ord.rename(columns={'numero de ordenes de proteccion': 'numero_ordenes_proteccion'}, inplace=True)
ord.año = clean_year(ord.año)

In [25]:
ord.trimestre = lower_tildes(ord.trimestre)
ord.provincia = lower_tildes(ord.provincia)
ord.isna().value_counts()

año    trimestre  provincia  numero_ordenes_proteccion
False  False      False      False                        3016
Name: count, dtype: int64

In [26]:
ord['numero_ordenes_proteccion'] = ord['numero_ordenes_proteccion'].astype(int)

In [27]:
ord.head()

,año,trimestre,provincia,numero_ordenes_proteccion
0,2009,primero,almeria,82
1,2009,primero,cadiz,312
2,2009,primero,cordoba,125
3,2009,primero,granada,163
4,2009,primero,huelva,138


In [28]:
ord.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3016 entries, 0 to 3015
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   año                        3016 non-null   int32 
 1   trimestre                  3016 non-null   object
 2   provincia                  3016 non-null   object
 3   numero_ordenes_proteccion  3016 non-null   int64 
dtypes: int32(1), int64(1), object(2)
memory usage: 82.6+ KB


### 4. LIMPIEZA ARCHIVO DISPOSITIVOS DE SEGUIMIENTO

In [29]:
disp = pd.read_csv('../data/dispositivos.csv')
disp.head()

,Año,Mes,Almería,Cádiz,Córdoba,Granada,Huelva,Jaén,Málaga,Sevilla,Huesca,Teruel,Zaragoza,Asturias,Illes Balears,Las Palmas,Santa Cruz de Tenerife,Cantabria,Ávila,Burgos,León,Palencia,Salamanca,Segovia,Soria,Valladolid,Zamora,Albacete,Ciudad Real,Cuenca,Guadalajara,Toledo,Barcelona,Girona,Lleida,Tarragona,Alicante/Alacant,Castellón/Castelló,Valencia/València,Badajoz,Cáceres,A Coruña,Lugo,Ourense,Pontevedra,Madrid,Murcia,Navarra,Araba/Álava,Bizkaia,Gipuzkoa,La Rioja,Ceuta,Melilla
0,Año 2009,Agosto,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Año 2009,Septiembre,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Año 2009,Octubre,1.0,1.0,1.0,0.0,0.0,3.0,2.0,2.0,0.0,0.0,0.0,2.0,8.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,2.0,1.0,3.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,Año 2009,Noviembre,1.0,1.0,1.0,0.0,1.0,5.0,2.0,2.0,0.0,0.0,0.0,2.0,9.0,0.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,1.0,5.0,2.0,0.0,2.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,5.0,1.0,0.0,0.0,0.0
4,Año 2009,Diciembre,1.0,1.0,2.0,2.0,1.0,8.0,3.0,2.0,0.0,0.0,0.0,2.0,9.0,0.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,5.0,1.0,8.0,3.0,0.0,2.0,1.0,0.0,1.0,88.0,0.0,1.0,0.0,7.0,1.0,0.0,0.0,0.0


In [30]:
disp.isna().value_counts()

Año    Mes    Almería  Cádiz  Córdoba  Granada  Huelva  Jaén   Málaga  Sevilla  Huesca  Teruel  Zaragoza  Asturias  Illes Balears  Las Palmas  Santa Cruz de Tenerife  Cantabria  Ávila  Burgos  León   Palencia  Salamanca  Segovia  Soria  Valladolid  Zamora  Albacete  Ciudad Real  Cuenca  Guadalajara  Toledo  Barcelona  Girona  Lleida  Tarragona  Alicante/Alacant  Castellón/Castelló  Valencia/València  Badajoz  Cáceres  A Coruña  Lugo   Ourense  Pontevedra  Madrid  Murcia  Navarra  Araba/Álava  Bizkaia  Gipuzkoa  La Rioja  Ceuta  Melilla
False  False  False    False  False    False    False   False  False   False    False   False   False     False     False          False       False                   False      False  False   False  False     False      False    False  False       False   False     False        False   False        False   False      False   False   False      False             False               False              False    False    False     False  False    False     

In [31]:
disp.columns = unif_col(disp.columns)
disp.mes = lower_tildes(disp.mes)
disp.rename(columns={'ano': 'año'}, inplace=True)
disp.año = clean_year(disp.año)

In [32]:
# Añadimos columna de trimestre:
disp['trimestre'] = disp['mes'].apply(trimestre)

# Vamos a reordenar las columnas:
columnas = disp.columns.tolist()

# Extraemos la última columna de trimestre:
ultima_columna = columnas[-1]

# La movemos a la tercera posición para que tenga consistencia:
columnas.insert(2, columnas.pop(-1))

# Reordenamos:
disp = disp[columnas]

In [33]:
disp.head()

,año,mes,trimestre,almeria,cadiz,cordoba,granada,huelva,jaen,malaga,sevilla,huesca,teruel,zaragoza,asturias,illes balears,las palmas,santa cruz de tenerife,cantabria,avila,burgos,leon,palencia,salamanca,segovia,soria,valladolid,zamora,albacete,ciudad real,cuenca,guadalajara,toledo,barcelona,girona,lleida,tarragona,alicante/alacant,castellon/castello,valencia/valencia,badajoz,caceres,a coruna,lugo,ourense,pontevedra,madrid,murcia,navarra,araba/alava,bizkaia,gipuzkoa,la rioja,ceuta,melilla
0,2009,agosto,tercero,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2009,septiembre,tercero,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2009,octubre,cuarto,1.0,1.0,1.0,0.0,0.0,3.0,2.0,2.0,0.0,0.0,0.0,2.0,8.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,2.0,1.0,3.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,2009,noviembre,cuarto,1.0,1.0,1.0,0.0,1.0,5.0,2.0,2.0,0.0,0.0,0.0,2.0,9.0,0.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,1.0,5.0,2.0,0.0,2.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,5.0,1.0,0.0,0.0,0.0
4,2009,diciembre,cuarto,1.0,1.0,2.0,2.0,1.0,8.0,3.0,2.0,0.0,0.0,0.0,2.0,9.0,0.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,5.0,1.0,8.0,3.0,0.0,2.0,1.0,0.0,1.0,88.0,0.0,1.0,0.0,7.0,1.0,0.0,0.0,0.0


In [34]:
col_int = disp.columns[3:]

disp[col_int] = disp[col_int].astype(int)

In [35]:
disp = pd.melt(disp, id_vars=['año', 'mes', 'trimestre'], 
                var_name='provincia', value_name='total_dispositivos')

In [36]:
disp.head()

,año,mes,trimestre,provincia,total_dispositivos
0,2009,agosto,tercero,almeria,1
1,2009,septiembre,tercero,almeria,1
2,2009,octubre,cuarto,almeria,1
3,2009,noviembre,cuarto,almeria,1
4,2009,diciembre,cuarto,almeria,1


In [37]:
disp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8840 entries, 0 to 8839
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   año                 8840 non-null   int32 
 1   mes                 8840 non-null   object
 2   trimestre           8840 non-null   object
 3   provincia           8840 non-null   object
 4   total_dispositivos  8840 non-null   int64 
dtypes: int32(1), int64(1), object(3)
memory usage: 310.9+ KB


### 5. LIMPIEZA ARCHIVO CSV MENORES VÍCTIMAS MORTALES DE VIOLENCIA DE GÉNERO:

In [38]:
men = pd.read_csv('../data/menores.csv')
men.head()

,Año,Trimestre,Tramo de edad de la víctima,Padre biológico,Suicidio,Almería,Cádiz,Jaén,Málaga,Teruel,Asturias,Illes Balears,Santa Cruz de Tenerife,Cantabria,Valladolid,Ciudad Real,Barcelona,Alicante/Alacant,Castellón/Castelló,Valencia/València,A Coruña,Pontevedra,Madrid,Murcia
0,Año 2013,Primero,3-4 años,Sí,No hubo tentativa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Año 2013,Segundo,<1 año,Sí,Suicidio consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Año 2013,Segundo,5-6 años,Sí,Suicidio consumado,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Año 2013,Segundo,11-12 años,Sí,Suicidio consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Año 2013,Tercero,5-6 años,Sí,No hubo tentativa,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
men.columns = unif_col(men.columns)
men.rename(columns={'padre biologico': 'padre_biologico'}, inplace=True)
men.rename(columns={'ano': 'año'}, inplace=True)
men.rename(columns={'tramo de edad de la victima': 'edad'}, inplace=True)

men['padre_biologico'] = lower_tildes(men['padre_biologico'])
men.trimestre = lower_tildes(men.trimestre)
men.suicidio = lower_tildes(men.suicidio)
men.edad = eliminar_alfa(men.edad)
men.año = clean_year(men.año)
men.fillna(0, inplace = True)

In [40]:
men.suicidio.unique()

array(['no hubo tentativa', 'suicidio consumado',
       'tentativa no consumada'], dtype=object)

In [41]:
suicidio = {'no hubo tentativa': 'no',
               'suicidio consumado': 'si',
               'tentativa no consumada': 'tentativa'}

men.suicidio = men.suicidio.map(suicidio)

In [42]:
men.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   año                     48 non-null     int32  
 1   trimestre               48 non-null     object 
 2   edad                    48 non-null     object 
 3   padre_biologico         48 non-null     object 
 4   suicidio                48 non-null     object 
 5   almeria                 48 non-null     float64
 6   cadiz                   48 non-null     float64
 7   jaen                    48 non-null     float64
 8   malaga                  48 non-null     float64
 9   teruel                  48 non-null     float64
 10  asturias                48 non-null     float64
 11  illes balears           48 non-null     float64
 12  santa cruz de tenerife  48 non-null     float64
 13  cantabria               48 non-null     float64
 14  valladolid              48 non-null     floa

In [43]:
col_int = men.columns[5:]

men[col_int] = men[col_int].astype(int)

In [44]:
men = pd.melt(men, id_vars=['año', 'trimestre', 'edad', 'padre_biologico', 'suicidio'], 
                var_name='provincia', value_name='total_menores_vict_mortales')

In [45]:
men.head()

,año,trimestre,edad,padre_biologico,suicidio,provincia,total_menores_vict_mortales
0,2013,primero,3-4,si,no,almeria,0
1,2013,segundo,<1,si,si,almeria,0
2,2013,segundo,5-6,si,si,almeria,0
3,2013,segundo,11-12,si,si,almeria,0
4,2013,tercero,5-6,si,no,almeria,0


### 6. LIMPIEZA ARCHIVO CSV VÍCTIMAS MORTALES POR VIOLENCIA DE GÉNERO

In [46]:
vic = pd.read_csv('../data/victimas_mortales.csv')
vic.head()

,Año,Mes,Pareja - Expareja,Convivencia,Denuncia agresor,Suicidio,Tramo de edad Agresor,Tramo de edad Víctima,País de nacimiento Agresor,País de nacimiento Víctima,Almería,Cádiz,Córdoba,Granada,Huelva,Jaén,Málaga,Sevilla,Huesca,Teruel,Zaragoza,Asturias,Illes Balears,Las Palmas,Santa Cruz de Tenerife,Cantabria,Ávila,Burgos,León,Palencia,Salamanca,Segovia,Soria,Valladolid,Zamora,Albacete,Ciudad Real,Cuenca,Guadalajara,Toledo,Barcelona,Girona,Lleida,Tarragona,Alicante/Alacant,Castellón/Castelló,Valencia/València,Badajoz,Cáceres,A Coruña,Lugo,Ourense,Pontevedra,Madrid,Murcia,Navarra,Araba/Álava,Bizkaia,Gipuzkoa,La Rioja,Ceuta,Melilla
0,Año 2003,Enero,Expareja,No,No consta denuncia,No hubo tentativa,No consta,31-40 años,España,España,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Año 2003,Enero,Pareja,Sí,No consta denuncia,No hubo tentativa,31-40 años,31-40 años,Otro país,España,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2,Año 2003,Enero,Pareja,Sí,No consta denuncia,No hubo tentativa,51-60 años,41-50 años,España,España,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Año 2003,Enero,Pareja,Sí,No consta denuncia,No hubo tentativa,No consta,21-30 años,No consta,No consta,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Año 2003,Enero,Pareja,Sí,No consta denuncia,No hubo tentativa,No consta,31-40 años,España,España,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# Formateamos sin minusculas ni tildes y modificamos nulos:
vic.fillna(0, inplace = True)
vic = vic.apply(lower_tildes)
vic.columns = unif_col(vic.columns)
vic.head()

,ano,mes,pareja - expareja,convivencia,denuncia agresor,suicidio,tramo de edad agresor,tramo de edad victima,pais de nacimiento agresor,pais de nacimiento victima,almeria,cadiz,cordoba,granada,huelva,jaen,malaga,sevilla,huesca,teruel,zaragoza,asturias,illes balears,las palmas,santa cruz de tenerife,cantabria,avila,burgos,leon,palencia,salamanca,segovia,soria,valladolid,zamora,albacete,ciudad real,cuenca,guadalajara,toledo,barcelona,girona,lleida,tarragona,alicante/alacant,castellon/castello,valencia/valencia,badajoz,caceres,a coruna,lugo,ourense,pontevedra,madrid,murcia,navarra,araba/alava,bizkaia,gipuzkoa,la rioja,ceuta,melilla
0,ano 2003,enero,expareja,no,no consta denuncia,no hubo tentativa,no consta,31-40 anos,espana,espana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ano 2003,enero,pareja,si,no consta denuncia,no hubo tentativa,31-40 anos,31-40 anos,otro pais,espana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,ano 2003,enero,pareja,si,no consta denuncia,no hubo tentativa,51-60 anos,41-50 anos,espana,espana,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ano 2003,enero,pareja,si,no consta denuncia,no hubo tentativa,no consta,21-30 anos,no consta,no consta,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ano 2003,enero,pareja,si,no consta denuncia,no hubo tentativa,no consta,31-40 anos,espana,espana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
# Renombramos las siguientes columnas:
vic.rename(columns={'ano': 'año'}, inplace=True)
vic.rename(columns={'tramo de edad agresor': 'edad_agresor'}, inplace=True)
vic.rename(columns={'tramo de edad victima': 'edad_victima'}, inplace=True)
vic.rename(columns={'pareja - expareja': 'pareja'}, inplace=True)
vic.rename(columns={'ano': 'año'}, inplace=True)
vic.rename(columns={'pais de nacimiento agresor': 'nacimiento_agresor'}, inplace=True)
vic.rename(columns={'pais de nacimiento victima': 'nacimiento_victima'}, inplace=True)
vic.rename(columns={'denuncia agresor': 'denuncia'}, inplace=True)

In [49]:
# Transformamos valores en las filas:
vic['edad_agresor'] = eliminar_alfa(vic['edad_agresor'])
vic['edad_victima'] = eliminar_alfa(vic['edad_victima'])
vic.año = clean_year(vic.año)
vic.suicidio = vic.suicidio.map(suicidio)


In [50]:
vic.pareja.unique()

array(['expareja', 'pareja', 'pareja en fase de separacion'], dtype=object)

In [51]:
pareja = {'expareja': 'no',
          'pareja': 'si',
          'pareja en fase de separacion': 'en_separacion'}

vic.pareja = vic.pareja.map(pareja)

In [52]:
vic.denuncia.unique()

array(['no consta denuncia', 'no habia denuncia', 'habia denuncia'],
      dtype=object)

In [53]:
denuncia = {'no consta denuncia': 'desconocido',
          'no habia denuncia': 'no',
          'habia denuncia': 'no'}

vic.denuncia = vic.denuncia.map(denuncia)

In [54]:
vic['edad_victima'].unique()

array(['31-40', '41-50', '21-30', '71-84', '', '61-70', '51-60', '18-20',
       '>=85', '<16', '16-17'], dtype=object)

In [55]:
vic['edad_victima'] = vic['edad_victima'].replace('', 'desconocido')
vic['edad_agresor'] = vic['edad_agresor'].replace('', 'desconocido')

In [56]:
vic['nacimiento_victima'].unique()

array(['espana', 'no consta', 'otro pais'], dtype=object)

In [57]:
nacim = {'espana': 'españa',
         'no consta': 'desconocido',
         'otro pais': 'otro'}


vic['nacimiento_victima'] = vic['nacimiento_victima'].map(nacim)
vic['nacimiento_agresor'] = vic['nacimiento_agresor'].map(nacim)

In [58]:
# Añadimos columna de trimestre:
vic['trimestre'] = vic['mes'].apply(trimestre)

# Vamos a reordenar las columnas:
columnas = vic.columns.tolist()

# Extraemos la última columna de trimestre:
ultima_columna = columnas[-1]
ultima_columna
# La movemos a la tercera posición para que tenga consistencia:
columnas.insert(2, columnas.pop(-1))

# Reordenamos:
vic = vic[columnas]

In [59]:
vic = pd.melt(vic, id_vars=['año', 'mes', 'trimestre', 'pareja', 'convivencia',
                              'denuncia', 'suicidio', 'edad_agresor', 'edad_victima',
                              'nacimiento_agresor', 'nacimiento_victima'], 
                var_name='provincia', value_name='total_victimas_mortales')

In [60]:
vic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63856 entries, 0 to 63855
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   año                      63856 non-null  int32 
 1   mes                      63856 non-null  object
 2   trimestre                63856 non-null  object
 3   pareja                   63856 non-null  object
 4   convivencia              63856 non-null  object
 5   denuncia                 63856 non-null  object
 6   suicidio                 63856 non-null  object
 7   edad_agresor             63856 non-null  object
 8   edad_victima             63856 non-null  object
 9   nacimiento_agresor       63856 non-null  object
 10  nacimiento_victima       63856 non-null  object
 11  provincia                63856 non-null  object
 12  total_victimas_mortales  63856 non-null  object
dtypes: int32(1), object(12)
memory usage: 6.1+ MB


In [61]:
# Cambiamos el tipo de dato de las columnas que tienen los totales

vic['total_victimas_mortales'] = vic['total_victimas_mortales'].apply(pd.to_numeric, errors='coerce')
vic['total_victimas_mortales'] = vic['total_victimas_mortales'].astype(int)

In [62]:
vic.head()

,año,mes,trimestre,pareja,convivencia,denuncia,suicidio,edad_agresor,edad_victima,nacimiento_agresor,nacimiento_victima,provincia,total_victimas_mortales
0,2003,enero,primero,no,no,desconocido,no,desconocido,31-40,españa,españa,almeria,0
1,2003,enero,primero,si,si,desconocido,no,31-40,31-40,otro,españa,almeria,0
2,2003,enero,primero,si,si,desconocido,no,51-60,41-50,españa,españa,almeria,1
3,2003,enero,primero,si,si,desconocido,no,desconocido,21-30,desconocido,desconocido,almeria,0
4,2003,enero,primero,si,si,desconocido,no,desconocido,31-40,españa,españa,almeria,0


### 7. LIMPIEZA ARCHIVO CSV SENTENCIAS

In [63]:
sent = pd.read_csv('../data/sentencias.csv', delimiter='\t')
sent

,Tipo de indicador,Tipo de dato,Periodo,Total
0,Condenados,Dato base,2022,36.161
1,Condenados,Dato base,2021,33.068
2,Condenados,Dato base,2020,25.436
3,Condenados,Dato base,2019,30.495
4,Condenados,Dato base,2018,27.972
5,Condenados,Dato base,2017,27.202
6,Condenados,Dato base,2016,25.959
7,Condenados,Dato base,2015,24.265
8,Absueltos,Dato base,2022,NaN
9,Absueltos,Dato base,2021,4.263


In [64]:
sent = sent.dropna()

In [65]:
sent = sent.apply(lower_tildes)
sent.columns = unif_col(sent.columns)

In [66]:
sent

,tipo de indicador,tipo de dato,periodo,total
0,condenados,dato base,2022,36.161
1,condenados,dato base,2021,33.068
2,condenados,dato base,2020,25.436
3,condenados,dato base,2019,30.495
4,condenados,dato base,2018,27.972
5,condenados,dato base,2017,27.202
6,condenados,dato base,2016,25.959
7,condenados,dato base,2015,24.265
9,absueltos,dato base,2021,4.263
10,absueltos,dato base,2020,3.489


In [67]:
# Renombramos las siguientes columnas:
sent.rename(columns={'tipo de indicador': 'indicador'}, inplace=True)
sent.rename(columns={'periodo': 'año'}, inplace=True)

In [68]:
sent = sent[['año', 'indicador', 'total']]
sent = sent.sort_values(by = 'año').reset_index(drop=True)

sent

,año,indicador,total
0,2015,condenados,24.265
1,2015,absueltos,6.768
2,2016,condenados,25.959
3,2016,absueltos,6.288
4,2017,condenados,27.202
5,2017,absueltos,6.181
6,2018,condenados,27.972
7,2018,absueltos,5.025
8,2019,condenados,30.495
9,2019,absueltos,5.149


In [69]:
sent['total'] = sent['total'].str.replace('.', '')
sent['total'] = pd.to_numeric(sent['total'], errors='coerce')

In [70]:
sent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   año        15 non-null     object
 1   indicador  15 non-null     object
 2   total      15 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 488.0+ bytes


In [71]:
sent.head()

,año,indicador,total
0,2015,condenados,24265
1,2015,absueltos,6768
2,2016,condenados,25959
3,2016,absueltos,6288
4,2017,condenados,27202


### 8. LIMPIEZA ARCHIVO CSV TIPOS DE VIOLENCIAS

In [72]:
tipos = pd.read_csv('../data/tipos.csv', encoding= 'utf-8', sep=';')
tipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2736 entries, 0 to 2735
Data columns (total 4 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Tipo de infracción                2736 non-null   object 
 1   Comunidades y ciudades autónomas  2736 non-null   object 
 2   Periodo                           2736 non-null   int64  
 3   Total                             2584 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 85.6+ KB


In [73]:
# Formateamos sin minusculas ni tildes y borramos nulos:
tipos = tipos.apply(lower_tildes)
tipos.columns = unif_col(tipos.columns)

In [74]:
tipos.isna().value_counts()

tipo de infraccion  comunidades y ciudades autonomas  periodo  total
False               False                             False    False    2736
Name: count, dtype: int64

In [75]:
tipos.dropna(how='any', inplace=True) 

In [76]:
tipos.rename(columns={'tipo de infraccion': 'tipo'}, inplace=True)
tipos.rename(columns={'comunidades y ciudades autonomas': 'comunidad'}, inplace=True)
tipos.rename(columns={'periodo': 'año'}, inplace=True)

In [77]:
tipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2736 entries, 0 to 2735
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tipo       2736 non-null   object
 1   comunidad  2736 non-null   object
 2   año        2736 non-null   object
 3   total      2736 non-null   object
dtypes: object(4)
memory usage: 85.6+ KB


In [78]:
tipos.tipo.unique()

array(['delitos', 'homicidio y sus formas', 'lesiones',
       'detenciones ilegales y secuestro', 'amenazas', 'coacciones',
       'torturas e integridad moral', 'agresiones sexuales',
       'abusos sexuales',
       'abusos y agresiones sexuales a menores de 16 anos',
       'allanamiento de morada', 'injurias', 'danos',
       'quebrantamiento de condena', 'otros delitos sin especificar',
       'faltas', 'faltas contra las personas',
       'otras faltas sin especificar'], dtype=object)

In [79]:
tipologia = {'delitos': 'delito', 
             'homicidio y sus formas': 'homicidio', 
             'lesiones': 'lesiones',
             'detenciones ilegales y secuestro': 'detencion_secuestro', 
             'amenazas': 'amenazas', 
             'coacciones': 'coacciones',
             'torturas e integridad moral': 'tortura_integridad_moral', 
             'agresiones sexuales': 'agresion_sexual',
             'abusos sexuales': 'abuso_sexual',
             'abusos y agresiones sexuales a menores de 16 anos': 'abuso_menores',
             'allanamiento de morada': 'allanamiento', 
             'injurias': 'injurias', 
             'danos': 'daños',
             'quebrantamiento de condena': 'quebrantamiento_condena', 
             'otros delitos sin especificar': 'otros',
             'faltas': 'faltas', 
             'faltas contra las personas': 'faltas',
             'otras faltas sin especificar': 'faltas'}

tipos.tipo = tipos.tipo.map(tipologia)

In [80]:
tipos.tipo.unique()

array(['delito', 'homicidio', 'lesiones', 'detencion_secuestro',
       'amenazas', 'coacciones', 'tortura_integridad_moral',
       'agresion_sexual', 'abuso_sexual', 'abuso_menores', 'allanamiento',
       'injurias', 'daños', 'quebrantamiento_condena', 'otros', 'faltas'],
      dtype=object)

In [81]:
tipos['total'].value_counts()

total
0.0      481
1.0      222
nan      152
2.0      146
3.0       88
        ... 
1.502      1
1.523      1
1.725      1
1.425      1
312.0      1
Name: count, Length: 617, dtype: int64

In [82]:
tipos = tipos.sort_values(by = 'año').reset_index(drop = True)

In [83]:
tipos.head()

,tipo,comunidad,año,total
0,abuso_sexual,melilla,2015,0.0
1,agresion_sexual,"asturias, principado de",2015,3.0
2,agresion_sexual,"balears, illes",2015,1.0
3,agresion_sexual,canarias,2015,1.0
4,agresion_sexual,cantabria,2015,0.0


In [84]:

tipos['total'] = tipos['total'].str.replace('.', '').replace('nan', float('nan'), regex=False)
tipos['total'] = pd.to_numeric(tipos['total'], errors='coerce')

tipos['total'].fillna(0, inplace=True)
tipos['total'] = tipos['total'].astype(int)


In [85]:
tipos.comunidad.unique()

array(['melilla', 'asturias, principado de', 'balears, illes', 'canarias',
       'cantabria', 'castilla y leon', 'castilla - la mancha', 'cataluna',
       'comunitat valenciana', 'extremadura', 'galicia',
       'madrid, comunidad de', 'murcia, region de',
       'navarra, comunidad foral de', 'pais vasco', 'rioja, la', 'aragon',
       'ceuta', 'andalucia'], dtype=object)

### 9. LIMPIEZA ARCHIVO CSV CONDENADOS POR VG POR EDAD Y CCAA

In [86]:
cond = pd.read_csv('../data/condenados_edad_ccaa.csv', encoding= 'utf-8', sep=';')
cond

,Comunidades y ciudades autónomas,Grupo de edad,Periodo,Total
0,Andalucía,Menos de 18 años,2022,17.0
1,Andalucía,Menos de 18 años,2021,19.0
2,Andalucía,Menos de 18 años,2020,25.0
3,Andalucía,Menos de 18 años,2019,34.0
4,Andalucía,Menos de 18 años,2018,22.0
...,...,...,...,...
755,Melilla,65 y más años,2019,0.0
756,Melilla,65 y más años,2018,2.0
757,Melilla,65 y más años,2017,3.0
758,Melilla,65 y más años,2016,3.0


In [87]:
cond.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 4 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Comunidades y ciudades autónomas  760 non-null    object 
 1   Grupo de edad                     760 non-null    object 
 2   Periodo                           760 non-null    int64  
 3   Total                             760 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 23.9+ KB


In [88]:
# Formateamos sin minusculas ni tildes:
cond = cond.apply(lower_tildes)
cond.columns = unif_col(cond.columns)

In [89]:
cond.head()

,comunidades y ciudades autonomas,grupo de edad,periodo,total
0,andalucia,menos de 18 anos,2022,17.0
1,andalucia,menos de 18 anos,2021,19.0
2,andalucia,menos de 18 anos,2020,25.0
3,andalucia,menos de 18 anos,2019,34.0
4,andalucia,menos de 18 anos,2018,22.0


In [90]:
cond.rename(columns={'comunidades y ciudades autonomas': 'comunidad',
                     'periodo': 'año',
                     'grupo de edad': 'edad'}, inplace=True)


In [91]:
cond['total'] = pd.to_numeric(cond['total'], errors='coerce')
cond['total'] = cond['total'].round().astype(int)

In [92]:
cond['edad'].unique()

array(['menos de 18 anos', 'de 18 a 24 anos', 'de 25 a 44 anos',
       'de 45 a 64 anos', '65 y mas anos'], dtype=object)

In [93]:
edad = {'menos de 18 anos': '<18',
        'de 18 a 24 anos': '18-24',
        'de 25 a 44 anos': '25-44',
        'de 45 a 64 anos': '45-64',
        '65 y mas anos': '>65'}

cond.edad = cond.edad.map(edad)

In [94]:
cond.comunidad.unique()

array(['andalucia', 'aragon', 'asturias, principado de', 'balears, illes',
       'canarias', 'cantabria', 'castilla y leon', 'castilla - la mancha',
       'cataluna', 'comunitat valenciana', 'extremadura', 'galicia',
       'madrid, comunidad de', 'murcia, region de',
       'navarra, comunidad foral de', 'pais vasco', 'rioja, la', 'ceuta',
       'melilla'], dtype=object)

In [95]:
cond = cond.sort_values(by = 'año').reset_index(drop = True)

In [96]:
cond.head()

,comunidad,edad,año,total
0,melilla,>65,2015,0
1,"balears, illes",25-44,2015,574
2,castilla - la mancha,25-44,2015,672
3,"madrid, comunidad de",45-64,2015,589
4,"rioja, la",<18,2015,0


### 10. LIMPIEZA ARCHIVOS CSV SALUD MENTAL 2006, 2012, 2017

Estos archivos recogen la media de salud mental en población mayor de 15 años. Los valores de la variable 'salud mental en población adulta' oscilan entre 0 y 12 puntos, de mejor salud mental a peor salud mental. El objetivo es unificarlos en un mismo archivo csv.

In [97]:
salud6 = pd.read_csv('../data/salud_mental2006.csv', encoding= 'Latin1', sep = '\t')
salud12 = pd.read_csv('../data/salud_mental2012.csv', encoding= 'Latin1', sep = '\t')
salud17 = pd.read_csv('../data/salud_mental2017.csv', encoding= 'Latin1', sep = '\t')


In [98]:
salud6.head()

,Sexo,Total Nacional,Comunidades y Ciudades Autónomas,Tipo de indicador,Total
0,VARONES,Total,Andalucía,Media,"1,13"
1,VARONES,Total,Aragón,Media,"0,85"
2,VARONES,Total,Asturias (Principado de),Media,"1,16"
3,VARONES,Total,Balears (Illes),Media,"1,40"
4,VARONES,Total,Canarias,Media,"1,42"


In [99]:
# Más adelante hemos tenido problemas con ese nulo, por lo que he regresado para subsanar

salud6.iloc[17]

Sexo                                        VARONES
Total Nacional                      Ceuta y Melilla
Comunidades y Ciudades Autónomas                NaN
Tipo de indicador                             Media
Total                                          1,21
Name: 17, dtype: object

In [100]:
salud6.loc[17, 'Comunidades y Ciudades Autónomas'] = 'Ceuta y Melilla'

In [101]:
salud6.iloc[35]

Sexo                                        MUJERES
Total Nacional                      Ceuta y Melilla
Comunidades y Ciudades Autónomas                NaN
Tipo de indicador                             Media
Total                                          2,02
Name: 35, dtype: object

In [102]:
salud6.loc[35, 'Comunidades y Ciudades Autónomas'] = 'Ceuta y Melilla'

In [103]:
# Limpiamos el archivo salud6 y le añadimos una columna con el año 2006:

salud6 = salud6.apply(lower_tildes)
salud6.columns = unif_col(salud6.columns)
salud6 = salud6.drop(['total nacional', 'tipo de indicador'], axis=1)
salud6.rename(columns={'comunidades y ciudades autonomas': 'comunidad', 
                       'total': 'media'}, inplace=True)
salud6['media'] = salud6['media'].str.replace(',', '.').astype(float)
salud6['año'] = 2006


In [104]:
# Gestionamos nulos para no tener problemas más adelante

salud6[salud6['comunidad'] == 'nan']

# En este momento toca subir arriba para subsanar (pertenecían a Ceuta y Melilla)

,sexo,comunidad,media,año


In [105]:
# Modificamos para que sea igual que en los otros dos archivos

salud6['sexo'].replace({'varones': 'hombres'}, inplace=True)

In [106]:
salud6.head()

,sexo,comunidad,media,año
0,hombres,andalucia,1.13,2006
1,hombres,aragon,0.85,2006
2,hombres,asturias (principado de),1.16,2006
3,hombres,balears (illes),1.40,2006
4,hombres,canarias,1.42,2006


In [107]:
salud6.comunidad.unique()

array(['andalucia', 'aragon', 'asturias (principado de)',
       'balears (illes)', 'canarias', 'cantabria', 'castilla y leon',
       'castilla-la mancha', 'cataluna', 'comunitat valenciana',
       'extremadura', 'galicia', 'madrid (comunidad de)',
       'murcia (region de)', 'navarra (comunidad foral de)', 'pais vasco',
       'rioja (la)', 'ceuta y melilla'], dtype=object)

In [108]:
salud6.comunidad = salud6.comunidad.apply(ccaa)

In [109]:
salud6.comunidad.unique()

array(['andalucia', 'aragon', 'asturias', 'balears', 'canarias',
       'cantabria', 'castilla y leon', 'castilla-la mancha', 'cataluna',
       'comunitat valenciana', 'extremadura', 'galicia', 'madrid',
       'murcia', 'navarra', 'pais vasco', 'rioja', 'ceuta y melilla'],
      dtype=object)

In [110]:
salud12.head()

,Sexo,Total Nacional,Comunidades y Ciudades Autónomas,Número medio,Total
0,HOMBRES,Total,Andalucía,Media,"1,27"
1,HOMBRES,Total,Aragón,Media,"0,85"
2,HOMBRES,Total,"Asturias, Principado de",Media,"1,46"
3,HOMBRES,Total,"Balears, Illes",Media,"1,16"
4,HOMBRES,Total,Canarias,Media,"1,83"


In [111]:
# Limpiamos el archivo salud12 y le añadimos una columna con el año 2012:

salud12 = salud12.apply(lower_tildes)
salud12.columns = unif_col(salud12.columns)
salud12 = salud12.drop(['total nacional', 'numero medio'], axis=1)
salud12.rename(columns={'comunidades y ciudades autonomas': 'comunidad', 'total': 'media'}, inplace=True)
salud12['media'] = salud12['media'].str.replace(',', '.').astype(float)
salud12['año'] = 2012

In [112]:
salud12.head()

,sexo,comunidad,media,año
0,hombres,andalucia,1.27,2012
1,hombres,aragon,0.85,2012
2,hombres,"asturias, principado de",1.46,2012
3,hombres,"balears, illes",1.16,2012
4,hombres,canarias,1.83,2012


In [113]:
salud17.head()

,Sexo,Total Nacional,Comunidades y Ciudades Autónomas,Indicador,Total
0,HOMBRES,Total,Andalucía,Media,"1,52"
1,HOMBRES,Total,Aragón,Media,"0,71"
2,HOMBRES,Total,"Asturias, Principado de",Media,"1,27"
3,HOMBRES,Total,"Balears, Illes",Media,"0,90"
4,HOMBRES,Total,Canarias,Media,"0,84"


In [114]:
# Limpiamos el archivo salud17 y le añadimos una columna con el año 2017:

salud17 = salud17.apply(lower_tildes)
salud17.columns = unif_col(salud17.columns)
salud17 = salud17.drop(['total nacional', 'indicador'], axis=1)
salud17.rename(columns={'comunidades y ciudades autonomas': 'comunidad', 'total': 'media'}, inplace=True)
salud17['media'] = salud17['media'].str.replace(',', '.').astype(float)
salud17['año'] = 2017

In [115]:
salud17.head()

,sexo,comunidad,media,año
0,hombres,andalucia,1.52,2017
1,hombres,aragon,0.71,2017
2,hombres,"asturias, principado de",1.27,2017
3,hombres,"balears, illes",0.90,2017
4,hombres,canarias,0.84,2017


In [116]:
salud = [salud6, salud12, salud17]
salud_mental = pd.concat(salud, ignore_index=True)
salud_mental.head()

,sexo,comunidad,media,año
0,hombres,andalucia,1.13,2006
1,hombres,aragon,0.85,2006
2,hombres,asturias,1.16,2006
3,hombres,balears,1.40,2006
4,hombres,canarias,1.42,2006


In [117]:
salud_mental.comunidad.unique()

array(['andalucia', 'aragon', 'asturias', 'balears', 'canarias',
       'cantabria', 'castilla y leon', 'castilla-la mancha', 'cataluna',
       'comunitat valenciana', 'extremadura', 'galicia', 'madrid',
       'murcia', 'navarra', 'pais vasco', 'rioja', 'ceuta y melilla',
       'asturias, principado de', 'balears, illes',
       'madrid, comunidad de', 'murcia, region de',
       'navarra, comunidad foral de', 'rioja, la', 'ceuta', 'melilla'],
      dtype=object)

In [118]:
# Aplicamos la función de normalizar los nombres de las comunidades autónomas a todos los df:

tipos, cond, salud_mental = comunidades([tipos, cond, salud_mental])


In [119]:
salud_mental.comunidad.unique()

array(['andalucia', 'aragon', 'asturias', 'baleares', 'canarias',
       'cantabria', 'castilla y leon', 'castilla la mancha', 'cataluña',
       'valencia', 'extremadura', 'galicia', 'madrid', 'navarra',
       'pais vasco', 'la rioja', 'melilla', 'region de murcia', 'ceuta'],
      dtype=object)

In [120]:
tipos.comunidad.unique()

array(['melilla', 'asturias', 'baleares', 'canarias', 'cantabria',
       'castilla y leon', 'castilla la mancha', 'cataluña', 'valencia',
       'extremadura', 'galicia', 'madrid', 'region de murcia', 'navarra',
       'pais vasco', 'la rioja', 'aragon', 'ceuta', 'andalucia'],
      dtype=object)

In [121]:
cond.comunidad.unique()

array(['melilla', 'baleares', 'castilla la mancha', 'madrid', 'la rioja',
       'asturias', 'cataluña', 'galicia', 'ceuta', 'aragon', 'pais vasco',
       'navarra', 'cantabria', 'castilla y leon', 'region de murcia',
       'canarias', 'extremadura', 'valencia', 'andalucia'], dtype=object)

### 11. LIMPIEZA DEL ARCHIVO CSV TASA DE ACTIVIDAD Y PARO

In [122]:
paro = pd.read_csv('../data/tasa_actividad_paro.csv', encoding= 'Latin1', sep = '\t')
paro.head()

,Sexo,Provincias,Tasas,Periodo,Total
0,Hombres,02 Albacete,Tasa de actividad,2023T3,"66,34"
1,Hombres,02 Albacete,Tasa de actividad,2023T2,"67,68"
2,Hombres,02 Albacete,Tasa de actividad,2023T1,"66,94"
3,Hombres,02 Albacete,Tasa de actividad,2022T4,"68,11"
4,Hombres,02 Albacete,Tasa de actividad,2022T3,"65,25"


- todo a minusculas y sin tildes
- limpiar tasas
- quitar lo numerico de provincias
- al terminar todos normalizar las provincias y añadir ccaa los df que no tengan
- limpiar periodo y dividir en año y trimestre
- limpiar total

In [123]:
paro = paro.apply(lower_tildes)
paro.columns = unif_col(paro.columns)
paro.rename(columns={'provincias': 'provincia', 
                     'periodo': 'año', 
                     'total': 'media',
                     'tasas': 'tasa'}, inplace=True)

In [124]:
paro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27144 entries, 0 to 27143
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sexo       27144 non-null  object
 1   provincia  27144 non-null  object
 2   tasa       27144 non-null  object
 3   año        27144 non-null  object
 4   media      27144 non-null  object
dtypes: object(5)
memory usage: 1.0+ MB


In [125]:
paro.tasa.unique()

array(['tasa de actividad', 'tasa de paro de la poblacion',
       'tasa de empleo de la poblacion'], dtype=object)

In [126]:
tasa = {'tasa de actividad': 'actividad', 
        'tasa de paro de la poblacion': 'paro', 
        'tasa de empleo de la poblacion': 'empleo'}

paro.tasa = paro.tasa.map(tasa)

In [127]:
paro.año.unique()

array(['2023t3', '2023t2', '2023t1', '2022t4', '2022t3', '2022t2',
       '2022t1', '2021t4', '2021t3', '2021t2', '2021t1', '2020t4',
       '2020t3', '2020t2', '2020t1', '2019t4', '2019t3', '2019t2',
       '2019t1', '2018t4', '2018t3', '2018t2', '2018t1', '2017t4',
       '2017t3', '2017t2', '2017t1', '2016t4', '2016t3', '2016t2',
       '2016t1', '2015t4', '2015t3', '2015t2', '2015t1', '2014t4',
       '2014t3', '2014t2', '2014t1', '2013t4', '2013t3', '2013t2',
       '2013t1', '2012t4', '2012t3', '2012t2', '2012t1', '2011t4',
       '2011t3', '2011t2', '2011t1', '2010t4', '2010t3', '2010t2',
       '2010t1', '2009t4', '2009t3', '2009t2', '2009t1', '2008t4',
       '2008t3', '2008t2', '2008t1', '2007t4', '2007t3', '2007t2',
       '2007t1', '2006t4', '2006t3', '2006t2', '2006t1', '2005t4',
       '2005t3', '2005t2', '2005t1', '2004t4', '2004t3', '2004t2',
       '2004t1', '2003t4', '2003t3', '2003t2', '2003t1', '2002t4',
       '2002t3', '2002t2', '2002t1'], dtype=object)

In [128]:
paro[['año', 'trimestre']] = paro['año'].str.split('t', expand=True)

In [129]:
paro.head()

,sexo,provincia,tasa,año,media,trimestre
0,hombres,02 albacete,actividad,2023,"66,34",3
1,hombres,02 albacete,actividad,2023,"67,68",2
2,hombres,02 albacete,actividad,2023,"66,94",1
3,hombres,02 albacete,actividad,2022,"68,11",4
4,hombres,02 albacete,actividad,2022,"65,25",3


In [130]:
trim = {'1': 'primero',
        '2': 'segundo',
        '3': 'tercero',
        '4': 'cuarto'}

paro.trimestre = paro.trimestre.map(trim)

In [131]:
paro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27144 entries, 0 to 27143
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sexo       27144 non-null  object
 1   provincia  27144 non-null  object
 2   tasa       27144 non-null  object
 3   año        27144 non-null  object
 4   media      27144 non-null  object
 5   trimestre  27144 non-null  object
dtypes: object(6)
memory usage: 1.2+ MB


In [132]:
paro.provincia = paro.provincia.apply(eliminar_num)
paro.provincia = paro.provincia.str.strip()

In [133]:
paro.provincia.unique()

array(['albacete', 'alicante/alacant', 'almeria', 'araba/alava',
       'asturias', 'avila', 'badajoz', 'balears, illes', 'barcelona',
       'bizkaia', 'burgos', 'caceres', 'cadiz', 'cantabria',
       'castellon/castello', 'ciudad real', 'cordoba', 'coruna, a',
       'cuenca', 'gipuzkoa', 'girona', 'granada', 'guadalajara', 'huelva',
       'huesca', 'jaen', 'leon', 'lleida', 'lugo', 'madrid', 'malaga',
       'murcia', 'navarra', 'ourense', 'palencia', 'palmas, las',
       'pontevedra', 'rioja, la', 'salamanca', 'santa cruz de tenerife',
       'segovia', 'sevilla', 'soria', 'tarragona', 'teruel', 'toledo',
       'valencia/valencia', 'valladolid', 'zamora', 'zaragoza', 'ceuta',
       'melilla'], dtype=object)

In [134]:
paro.media = paro.media.str.replace(',', '.')
paro.media = pd.to_numeric(paro.media, errors='coerce')

In [135]:
paro = paro.sort_values(by = 'año').reset_index(drop = True)

In [136]:
paro.head()

,sexo,provincia,tasa,año,media,trimestre
0,hombres,melilla,empleo,2002,62.64,primero
1,hombres,"rioja, la",actividad,2002,67.42,primero
2,hombres,"rioja, la",paro,2002,3.80,cuarto
3,hombres,"rioja, la",paro,2002,5.58,tercero
4,hombres,"rioja, la",paro,2002,6.50,segundo


In [137]:
paro[paro.isna().any(axis=1)]

,sexo,provincia,tasa,año,media,trimestre
210,hombres,melilla,paro,2002,NaN,segundo


In [138]:
paro = paro.drop(210).reset_index(drop = True)

In [139]:
paro.head()

,sexo,provincia,tasa,año,media,trimestre
0,hombres,melilla,empleo,2002,62.64,primero
1,hombres,"rioja, la",actividad,2002,67.42,primero
2,hombres,"rioja, la",paro,2002,3.80,cuarto
3,hombres,"rioja, la",paro,2002,5.58,tercero
4,hombres,"rioja, la",paro,2002,6.50,segundo


### 12. LIMPIEZA ARCHIVO CSV CONSUMO DE ALCOHOL

In [140]:
alcohol = pd.read_csv('../data/alcohol12.csv', encoding= 'Latin1', sep = '\t')
alcohol.head()

,Sexo,Total Nacional,Comunidades y Ciudades Autónomas,Frecuencia,Número medio,Total
0,HOMBRES,Total,Andalucía,Semanal (L-D),Media,"13,23"
1,HOMBRES,Total,Andalucía,Entre semana (L-J),Media,"7,94"
2,HOMBRES,Total,Andalucía,Fin de semana (V-D),Media,"20,28"
3,HOMBRES,Total,Aragón,Semanal (L-D),Media,"16,03"
4,HOMBRES,Total,Aragón,Entre semana (L-J),Media,"10,56"


- todo a minusculas y sin tildes
- ver si hay nulos como antes y añadir la ccaa que aparezca en total nacional a la comunidad
- limpiar frecuencia y normalizar
- eliminar media
- limpiar total
- aplicar funcion de normalizar ccaa


- arreglar csv de nivel educativo


In [141]:
alcohol = alcohol.apply(lower_tildes)
alcohol.columns = unif_col(alcohol.columns)
alcohol.rename(columns={'comunidades y ciudades autonomas': 'comunidad',  
                     'total': 'consumo_medio'}, inplace=True)

In [142]:
alcohol.isna().value_counts()

sexo   total nacional  comunidad  frecuencia  numero medio  consumo_medio
False  False           False      False       False         False            114
Name: count, dtype: int64

In [143]:
alcohol.frecuencia.unique()

array(['semanal (l-d)', 'entre semana (l-j)', 'fin de semana (v-d)'],
      dtype=object)

In [144]:
frec = {'semanal (l-d)': 'diario',
        'entre semana (l-j)': 'entre semana',
        'fin de semana (v-d)': 'fin de semana'}

alcohol.frecuencia = alcohol.frecuencia.map(frec)

In [145]:
alcohol.frecuencia.unique()

array(['diario', 'entre semana', 'fin de semana'], dtype=object)

In [146]:
alcohol = alcohol[['sexo', 'comunidad', 'frecuencia', 'consumo_medio']]

In [147]:
alcohol['consumo_medio'] = alcohol['consumo_medio'].str.replace(',', '.')
alcohol['consumo_medio'] = pd.to_numeric(alcohol['consumo_medio'], errors='coerce')

In [148]:
alcohol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   sexo           114 non-null    object 
 1   comunidad      114 non-null    object 
 2   frecuencia     114 non-null    object 
 3   consumo_medio  114 non-null    float64
dtypes: float64(1), object(3)
memory usage: 3.7+ KB


In [149]:
alcohol.head()

,sexo,comunidad,frecuencia,consumo_medio
0,hombres,andalucia,diario,13.23
1,hombres,andalucia,entre semana,7.94
2,hombres,andalucia,fin de semana,20.28
3,hombres,aragon,diario,16.03
4,hombres,aragon,entre semana,10.56


### 13. LIMPIEZA ARCHIVO CSV NIVEL EDUCATIVO

In [150]:
educ = pd.read_csv('../data/nivel.csv', encoding= 'Latin1', sep = '\t')
educ.head()

,Nivel de formación alcanzado,Sexo,Comunidades y Ciudades Autónomas,Periodo,Total
0,Analfabetos,Hombres,01 Andalucía,2023T3,"1,2"
1,Analfabetos,Hombres,01 Andalucía,2023T2,"1,4"
2,Analfabetos,Hombres,01 Andalucía,2023T1,"1,2"
3,Analfabetos,Hombres,01 Andalucía,2022T4,"1,5"
4,Analfabetos,Hombres,01 Andalucía,2022T3,"1,6"


In [151]:
educ = educ.apply(lower_tildes)
educ.columns = unif_col(educ.columns)
educ.rename(columns={'nivel de formacion alcanzado': 'nivel_educativo',  
                     'comunidades y ciudades autonomas': 'comunidad',
                     'periodo': 'año',
                     'total': 'media_poblacion'}, inplace=True)

In [152]:
educ.año.sort_values()

5186     2014t1
3938     2014t1
3899     2014t1
3860     2014t1
3821     2014t1
          ...  
9399     2023t3
10101    2023t3
7917     2023t3
7956     2023t3
0        2023t3
Name: año, Length: 10374, dtype: object

In [153]:
educ.año.unique()

array(['2023t3', '2023t2', '2023t1', '2022t4', '2022t3', '2022t2',
       '2022t1', '2021t4', '2021t3', '2021t2', '2021t1', '2020t4',
       '2020t3', '2020t2', '2020t1', '2019t4', '2019t3', '2019t2',
       '2019t1', '2018t4', '2018t3', '2018t2', '2018t1', '2017t4',
       '2017t3', '2017t2', '2017t1', '2016t4', '2016t3', '2016t2',
       '2016t1', '2015t4', '2015t3', '2015t2', '2015t1', '2014t4',
       '2014t3', '2014t2', '2014t1'], dtype=object)

In [154]:
educ[['año', 'trimestre']] = educ['año'].str.split('t', expand=True)


In [155]:
educ.head()

,nivel_educativo,sexo,comunidad,año,media_poblacion,trimestre
0,analfabetos,hombres,01 andalucia,2023,"1,2",3
1,analfabetos,hombres,01 andalucia,2023,"1,4",2
2,analfabetos,hombres,01 andalucia,2023,"1,2",1
3,analfabetos,hombres,01 andalucia,2022,"1,5",4
4,analfabetos,hombres,01 andalucia,2022,"1,6",3


In [156]:
educ.trimestre = educ.trimestre.map(trim)
educ.head()

,nivel_educativo,sexo,comunidad,año,media_poblacion,trimestre
0,analfabetos,hombres,01 andalucia,2023,"1,2",tercero
1,analfabetos,hombres,01 andalucia,2023,"1,4",segundo
2,analfabetos,hombres,01 andalucia,2023,"1,2",primero
3,analfabetos,hombres,01 andalucia,2022,"1,5",cuarto
4,analfabetos,hombres,01 andalucia,2022,"1,6",tercero


In [157]:
educ['comunidad'].unique()

array(['01 andalucia', '02 aragon', '03 asturias, principado de',
       '04 balears, illes', '05 canarias', '06 cantabria',
       '07 castilla y leon', '08 castilla - la mancha', '09 cataluna',
       '10 comunitat valenciana', '11 extremadura', '12 galicia',
       '13 madrid, comunidad de', '14 murcia, region de',
       '15 navarra, comunidad foral de', '16 pais vasco', '17 rioja, la',
       '18 ceuta', '19 melilla'], dtype=object)

In [158]:
educ.comunidad = educ.comunidad.apply(eliminar_num)
educ.comunidad = educ.comunidad.str.strip()

In [159]:
educ = educ.sort_values(by = 'año').reset_index(drop = True)

In [160]:
educ['media_poblacion'] = educ['media_poblacion'].str.replace(',', '.')
educ['media_poblacion'] = pd.to_numeric(educ['media_poblacion'], errors='coerce')

In [161]:
educ['nivel_educativo'].unique()

array(['primera etapa de educacion secundaria y similar',
       'educacion primaria', 'educacion superior',
       'estudios primarios incompletos', 'analfabetos',
       'segunda etapa de educacion secundaria con orientacion profesional (incluye educacion postsecundaria no superior)',
       'segunda etapa de educacion secundaria, con orientacion general'],
      dtype=object)

In [162]:
etapas = {'primera etapa de educacion secundaria y similar': 'educacion primaria',
       'educacion primaria': 'educacion primaria', 
       'educacion superior': 'educacion universitaria',
       'estudios primarios incompletos': 'primaria incompleta', 
       'analfabetos': 'sin estudios',
       'segunda etapa de educacion secundaria con orientacion profesional (incluye educacion postsecundaria no superior)': 'educacion secundaria',
       'segunda etapa de educacion secundaria, con orientacion general': 'educacion secundaria'}


educ['nivel_educativo'] = educ['nivel_educativo'].map(etapas)

In [163]:
educ[educ.isna().any(axis = 1)]

,nivel_educativo,sexo,comunidad,año,media_poblacion,trimestre
8425,sin estudios,hombres,ceuta,2021,NaN,cuarto
8438,sin estudios,hombres,ceuta,2021,NaN,tercero
8473,sin estudios,hombres,ceuta,2021,NaN,segundo
9123,sin estudios,hombres,ceuta,2022,NaN,primero


In [164]:
educ = educ.dropna(axis=0)

In [165]:
educ.head()

,nivel_educativo,sexo,comunidad,año,media_poblacion,trimestre
0,educacion primaria,hombres,melilla,2014,21.3,primero
1,educacion primaria,mujeres,galicia,2014,21.4,segundo
2,educacion primaria,mujeres,galicia,2014,20.9,tercero
3,educacion primaria,mujeres,galicia,2014,21.1,cuarto
4,educacion primaria,mujeres,extremadura,2014,13.4,primero


### 14. LIMPIEZA ARCHIVO CSV POBLACION TOTAL POR COMUNIDAD AUTÓNOMA


In [166]:
total_ccaa = pd.read_csv('../data/total_ccaa.csv', encoding= 'Latin1', sep = '\t')
total_ccaa = total_ccaa.drop(columns=['Nacional'])
total_ccaa = total_ccaa.dropna()
total_ccaa.head()

,Comunidades y Ciudades Autónomas,Sexo,Periodo,Total
30,01 Andalucía,Hombres,2021,4.173.339
31,01 Andalucía,Hombres,2020,4.170.605
32,01 Andalucía,Hombres,2019,4.147.167
33,01 Andalucía,Hombres,2018,4.133.898
34,01 Andalucía,Hombres,2017,4.133.835


In [167]:
total_ccaa = total_ccaa.apply(lower_tildes)
total_ccaa.columns = unif_col(total_ccaa.columns)
total_ccaa.rename(columns={'comunidades y ciudades autonomas': 'comunidad', 
                     'periodo': 'año'}, inplace=True)

In [168]:
total_ccaa = total_ccaa.sort_values(by = 'año').reset_index(drop = True)

In [169]:
total_ccaa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   comunidad  570 non-null    object
 1   sexo       570 non-null    object
 2   año        570 non-null    object
 3   total      570 non-null    object
dtypes: object(4)
memory usage: 17.9+ KB


In [170]:
total_ccaa['total'] = total_ccaa['total'].str.replace('.', '')
total_ccaa['total'] = pd.to_numeric(total_ccaa['total'], errors='coerce')

In [171]:
total_ccaa.head()

,comunidad,sexo,año,total
0,10 comunitat valenciana,hombres,2007,2432162
1,09 cataluna,mujeres,2007,3632332
2,09 cataluna,hombres,2007,3578176
3,08 castilla - la mancha,mujeres,2007,981636
4,08 castilla - la mancha,hombres,2007,995668


### 15. LIMPIEZA ARCHIVO CSV POBLACION TOTAL POR PROVINCIAS

In [172]:
total_prov = pd.read_csv('../data/total_provincias.csv', encoding= 'Latin1', sep = '\t')
total_prov.head()

,Provincias,Sexo,Periodo,Total
0,02 Albacete,Hombres,2021,193.205
1,02 Albacete,Hombres,2020,194.081
2,02 Albacete,Hombres,2019,193.911
3,02 Albacete,Hombres,2018,194.158
4,02 Albacete,Hombres,2017,194.743


In [173]:
total_prov = total_prov.apply(lower_tildes)
total_prov.columns = unif_col(total_prov.columns)
total_prov.rename(columns={'provincias': 'provincia', 
                     'periodo': 'año'}, inplace=True)

In [174]:
total_prov = total_prov.sort_values(by = 'año').reset_index(drop = True)

In [175]:
total_prov['total'] = total_prov['total'].str.replace('.', '')
total_prov['total'] = pd.to_numeric(total_prov['total'], errors='coerce')

In [176]:
total_prov.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   provincia  1560 non-null   object
 1   sexo       1560 non-null   object
 2   año        1560 non-null   object
 3   total      1560 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 48.9+ KB


In [177]:
total_prov.head()

,provincia,sexo,año,total
0,23 jaen,mujeres,2007,334199
1,14 cordoba,mujeres,2007,403009
2,"15 coruna, a",hombres,2007,544352
3,"15 coruna, a",mujeres,2007,588440
4,16 cuenca,hombres,2007,106633


#### NORMALIZAMOS TODAS LAS COLUMNAS DE PROVINCIA Y COMUNIDAD AUTÓNOMA DE TODOS LOS CSV:

In [178]:
# Normalizamos la columna de provincia de todos los dataframes:

provin = [denu, llam, ord, disp, men, vic, paro, total_prov]

provin_normalizado = provincias(provin)
denu, llam, ord, disp, men, vic, paro, total_prov = provin_normalizado

In [179]:
# Creamos la columna de comunidad donde solo hubiese la de provincia:
provins = [denu, llam, ord, disp, men, vic, paro]
comun = crear_ccaa(provins)
denu, llam, ord, disp, men, vic, paro = comun

In [180]:
# Normalizamos la columna de comunidad del resto de dataframes que la contienen:

comuni = [tipos, cond, alcohol, educ, total_ccaa]
normalizar = comunidades(comuni)
tipos, cond, alcohol, educ, total_ccaa = normalizar

#### CARGAMOS TODOS LOS DATAFRAMES LIMPIOS EN ARCHIVOS CSV:

In [186]:
denu.to_csv('../data_clean/portal_estadistico_vio_gen/denuncias.csv', index=False)
llam.to_csv('../data_clean/portal_estadistico_vio_gen/llamadas016.csv', index=False)
ord.to_csv('../data_clean/portal_estadistico_vio_gen/ordenes_prot.csv', index=False)
disp.to_csv('../data_clean/portal_estadistico_vio_gen/dispositivos.csv', index=False)
men.to_csv('../data_clean/portal_estadistico_vio_gen/menores.csv', index=False)
vic.to_csv('../data_clean/portal_estadistico_vio_gen/victimas_mortales.csv', index=False)
sent.to_csv('../data_clean/ine/sentencias.csv', index=False)
tipos.to_csv('../data_clean/ine/tipos_violencias.csv', index=False)
cond.to_csv('../data_clean/ine/condenados_edad.csv', index=False)
salud_mental.to_csv('../data_clean/ine/salud_mental.csv', index=False)
paro.to_csv('../data_clean/ine/tasa_actividad.csv', index=False)
alcohol.to_csv('../data_clean/ine/alcohol.csv', index=False)
educ.to_csv('../data_clean/ine/nivel_educativo.csv', index=False)
total_ccaa.to_csv('../data_clean/ine/total_ccaa.csv', index=False)
total_prov.to_csv('../data_clean/ine/total_prov.csv', index=False)

In [187]:
denu.head()

,año,trimestre,origen_denuncia,provincia,comunidad,total_denuncias
0,2009,primero,victima,almeria,andalucia,40
1,2009,primero,familiar,almeria,andalucia,1
2,2009,primero,policia_victima,almeria,andalucia,419
3,2009,primero,policia_familiar,almeria,andalucia,7
4,2009,primero,intervencion_policial_directa,almeria,andalucia,82
